In [ ]:
## Collecte de données

Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :

1.  Créer un dossier appelé *images*.
2.  Télécharger les images sous licence ouverte dans le dossier *images* (minimum 100
    images).
3.  Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image (.jpeg,
    .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.),
    date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations [Exif](https://en.wikipedia.org/wiki/Exif) présentes dans les fichiers d'images.

In [ ]:
import os

# Spécifiez le chemin du dossier que vous souhaitez créer
dossier = 'images'

# Créez le dossier
os.makedirs(dossier, exist_ok=True)

print(f"Le dossier '{dossier}' a été créé avec succès.")


In [ ]:
!pip install SPARQLWrapper

In [ ]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Initialiser le wrapper SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Définir la requête SPARQL pour obtenir des images de monuments
sparql.setQuery("""
SELECT ?monument ?monumentLabel ?image WHERE {
  ?monument wdt:P31 wd:Q839954.  # Monument
  ?monument wdt:P18 ?image.      # Image
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)

try:
    # Exécuter la requête et récupérer les résultats en JSON
    results = sparql.query().convert()
except Exception as e:
    print(f"❌ Erreur lors de l'exécution de la requête SPARQL : {e}")
    exit(1)

# Liste des images à télécharger avec des indices de 1 à 100
images = [(i + 1, result["image"]["value"]) for i, result in enumerate(results["results"]["bindings"])]

# Définition d'un User-Agent personnalisé
HEADERS = {
    "User-Agent": "MonScript/1.0 (mailto:mohamedguef@gmail.com)"  # Remplace par ton email pour respecter la politique
}

def download_image(data):
    index, image_url = data
    try:
        # Nom fixe sous le format image_1.jpg, image_2.jpg, ..., image_100.jpg
        image_name = f"image_{index}.jpg"
        image_path = os.path.join(IMAGE_DIR, image_name)

        # Télécharger l'image avec un User-Agent correct
        response = requests.get(image_url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Vérifie si la requête a réussi
        
        # Sauvegarder l'image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ Téléchargé : {image_name}")

    except requests.RequestException as e:
        print(f"⚠️ Erreur lors du téléchargement de {image_url} : {e}")
    except Exception as e:
        print(f"⚠️ Erreur inattendue : {e}")

# Téléchargement des images en parallèle
MAX_THREADS = 10  # Ajuste ce nombre en fonction de ta connexion et des performances

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, images)

print("✅ Tous les téléchargements sont terminés !")


In [ ]:
!pip install Pillow ExifRead

In [ ]:
!pip install tqdm

In [ ]:
import os
import json
from PIL import Image, UnidentifiedImageError
import exifread
from tqdm import tqdm  # Barre de progression

# Définir le dossier des images et le fichier de sortie JSON
IMAGE_DIR = "images"
METADATA_FILE = os.path.join(IMAGE_DIR, "metadata.json")

# Vérifier que le dossier "images" existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable. Vérifiez que les images sont bien téléchargées.")
    exit(1)

# Liste pour stocker les métadonnées de toutes les images
metadata_list = []

# Liste des fichiers images triés
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Parcourir les images et extraire les métadonnées
for image_name in tqdm(image_files, desc="📷 Extraction des métadonnées"):
    image_path = os.path.join(IMAGE_DIR, image_name)

    try:
        # Vérifier si l'image est corrompue en tentant de l'ouvrir
        with Image.open(image_path) as img:
            img.verify()  # Vérifie l'intégrité de l'image sans la charger
            img = Image.open(image_path)  # Recharge l'image pour la lecture des métadonnées
            width, height = img.size
            format_ = img.format
            orientation = 'Portrait' if height > width else 'Paysage' if width > height else 'Carré'

        # Vérifier si c'est un PNG (pas d'Exif)
        if format_.upper() == "PNG":
            exif_data = "Non disponible (format PNG)"
        else:
            # Extraire les métadonnées Exif avec exifread
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal", details=False)

            # Filtrer les champs problématiques
            exif_data = {
                "Date de création": str(tags.get("EXIF DateTimeOriginal", "Inconnu")),
                "Modèle d'appareil": str(tags.get("Image Model", "Inconnu")),
                "Marque d'appareil": str(tags.get("Image Make", "Inconnu")),
                "ISO": str(tags.get("EXIF ISOSpeedRatings", "Inconnu")),
                "Temps d'exposition": str(tags.get("EXIF ExposureTime", "Inconnu")),
                "Ouverture (f)": str(tags.get("EXIF FNumber", "Inconnu")),
                "Longueur focale": str(tags.get("EXIF FocalLength", "Inconnu")),
                "GPS Latitude": str(tags.get("GPS GPSLatitude", "Non disponible")),
                "GPS Longitude": str(tags.get("GPS GPSLongitude", "Non disponible")),
            }

            # Supprimer les champs corrompus (évite les erreurs "Possibly corrupted field")
            exif_data = {key: value for key, value in exif_data.items() if "Possibly corrupted" not in value}

        # Ajouter les métadonnées à la liste
        metadata_list.append({
            'Nom du fichier': image_name,
            'Taille': {'Largeur': width, 'Hauteur': height},
            'Format': format_,
            'Orientation': orientation,
            'Exif': exif_data
        })

    except UnidentifiedImageError:
        print(f"⚠️ Image illisible : {image_name} (corrompue ou format inconnu)")
    except Exception as e:
        print(f"⚠️ Erreur inattendue sur {image_name} : {e}")

# Enregistrer toutes les métadonnées dans un fichier JSON
try:
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"✅ Métadonnées enregistrées dans '{METADATA_FILE}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")


In [ ]:
pip install opencv-python

In [4]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import json

# Fonction pour extraire les couleurs dominantes
def get_dominant_colors(image_path, k=3):
    """
    Extrait les k couleurs dominantes d'une image en utilisant K-Means.
    :param image_path: Chemin de l'image
    :param k: Nombre de couleurs dominantes à extraire
    :return: Liste des couleurs dominantes au format RGB
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Impossible de lire l'image : {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB
    pixels = image.reshape(-1, 3)  # Redimensionner en tableau 2D (N pixels x 3 canaux)
    kmeans = KMeans(n_clusters=k)  # Appliquer K-Means
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)  # Extraire les couleurs dominantes
    return colors.tolist()  # Convertir en liste pour JSON

# Dossier contenant les images
IMAGE_DIR = "images"

# Vérifier que le dossier existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable.")
    exit(1)

# Dictionnaire pour stocker les couleurs dominantes de chaque image
dominant_colors_data = {}

# Parcourir toutes les images du dossier
for image_name in os.listdir(IMAGE_DIR):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers images
        image_path = os.path.join(IMAGE_DIR, image_name)
        print(f"🔍 Traitement de l'image : {image_name}")

        # Extraire les couleurs dominantes
        dominant_colors = get_dominant_colors(image_path, k=3)  # Extraire 3 couleurs dominantes
        if dominant_colors:
            dominant_colors_data[image_name] = dominant_colors
            print(f"✅ Couleurs dominantes pour {image_name} : {dominant_colors}")
        else:
            print(f"⚠️ Aucune couleur dominante trouvée pour {image_name}")

# Enregistrer les résultats dans un fichier JSON
output_file = os.path.join(IMAGE_DIR, "dominant_colors.json")
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dominant_colors_data, f, indent=4, ensure_ascii=False)
    print(f"✅ Couleurs dominantes enregistrées dans '{output_file}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")

🔍 Traitement de l'image : image_71.jpg
✅ Couleurs dominantes pour image_71.jpg : [[197, 197, 190], [48, 46, 21], [136, 125, 103]]
🔍 Traitement de l'image : image_82.jpg
✅ Couleurs dominantes pour image_82.jpg : [[116, 116, 75], [190, 185, 141], [48, 51, 28]]
🔍 Traitement de l'image : image_76.jpg
✅ Couleurs dominantes pour image_76.jpg : [[27, 25, 33], [209, 205, 208], [109, 97, 104]]
🔍 Traitement de l'image : image_24.jpg
✅ Couleurs dominantes pour image_24.jpg : [[186, 185, 187], [217, 217, 218], [132, 132, 132]]
🔍 Traitement de l'image : image_58.jpg
✅ Couleurs dominantes pour image_58.jpg : [[171, 167, 159], [100, 151, 210], [97, 99, 83]]
🔍 Traitement de l'image : image_38.jpg
✅ Couleurs dominantes pour image_38.jpg : [[109, 132, 102], [47, 66, 42], [178, 190, 193]]
🔍 Traitement de l'image : image_88.jpg
✅ Couleurs dominantes pour image_88.jpg : [[72, 61, 52], [186, 170, 159], [24, 97, 157]]
🔍 Traitement de l'image : image_53.jpg
✅ Couleurs dominantes pour image_53.jpg : [[73, 58, 

libpng warning: iCCP: known incorrect sRGB profile


✅ Couleurs dominantes pour image_35.jpg : [[48, 51, 36], [137, 112, 71], [195, 194, 213]]
🔍 Traitement de l'image : image_26.jpg
✅ Couleurs dominantes pour image_26.jpg : [[147, 140, 90], [70, 75, 39], [220, 205, 185]]
🔍 Traitement de l'image : image_32.jpg
✅ Couleurs dominantes pour image_32.jpg : [[219, 221, 226], [163, 141, 111], [111, 91, 68]]
🔍 Traitement de l'image : image_64.jpg
✅ Couleurs dominantes pour image_64.jpg : [[202, 186, 166], [63, 55, 50], [108, 179, 246]]
🔍 Traitement de l'image : image_95.jpg
⚠️ Impossible de lire l'image : images/image_95.jpg
⚠️ Aucune couleur dominante trouvée pour image_95.jpg
🔍 Traitement de l'image : image_84.jpg
✅ Couleurs dominantes pour image_84.jpg : [[215, 241, 243], [38, 49, 17], [142, 133, 34]]
🔍 Traitement de l'image : image_14.jpg
✅ Couleurs dominantes pour image_14.jpg : [[62, 61, 51], [249, 250, 250], [116, 115, 111]]
🔍 Traitement de l'image : image_100.jpg
✅ Couleurs dominantes pour image_100.jpg : [[139, 135, 96], [237, 235, 234],

In [10]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 12.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.3 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 29.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 32.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.0 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 34.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [14]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import json
from PIL import UnidentifiedImageError

# Charger le modèle ResNet50 pré-entraîné
model = ResNet50(weights='imagenet')

def classify_image(image_path):
    try:
        # Charger et prétraiter l'image
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Prédire les classes
        preds = model.predict(x)
        return decode_predictions(preds, top=3)[0]

    except UnidentifiedImageError:
        print(f"Erreur : Le fichier {image_path} n'est pas une image valide.")
        return []

# Liste pour stocker les prédictions de toutes les images
predictions_list = []

# Parcourir toutes les images dans le dossier 'images'
for file_name in os.listdir('images'):
    if file_name.startswith('image_') and file_name.endswith('.jpg'):
        image_path = os.path.join('images', file_name)

        # Classifier l'image
        predictions = classify_image(image_path)

        # Ajouter les prédictions à la liste si l'image est valide
        if predictions:
            predictions_list.append({
                'file_name': file_name,
                'predictions': [{'label': pred[1], 'score': float(pred[2])} for pred in predictions]
            })

        print(f"Prédictions pour {file_name}: {predictions}")

# Enregistrer toutes les prédictions dans un fichier JSON
with open('images/predictions.json', 'w') as f:
    json.dump(predictions_list, f, indent=4)

print("Prédictions enregistrées dans 'images/predictions.json'")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 986ms/step
Prédictions pour image_71.jpg: [('n02788148', 'bannister', np.float32(0.19757968)), ('n03888605', 'parallel_bars', np.float32(0.15790643)), ('n03535780', 'horizontal_bar', np.float32(0.123370714))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prédictions pour image_82.jpg: [('n03743016', 'megalith', np.float32(0.9820999)), ('n04326547', 'stone_wall', np.float32(0.0068500703)), ('n03891251', 'park_bench', np.float32(0.00093843864))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Prédictions pour image_76.jpg: [('n03733281', 'maze', np.float32(0.4356573)), ('n04258138', 'solar_dish', np.float32(0.07088651)), ('n04044716', 'radio_telescope', np.float32(0.0573914))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Prédictions pour image_24.jpg: [('n03924679', 'photocopier', np.float32(0.26402122)), ('n03291819', 'envelope', np.float32(0.12348314)), ('n04070727', 'refrigerator', np.float32(0.07831547))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Prédictions pour image_58.jpg: [('n09